# Imposing parameter links: Tortuosity and Equality

In [1]:
# load the necessary modules
from microstruktur.signal_models import cylinder_models, gaussian_models
from microstruktur.core import modeling_framework
from microstruktur.acquisition_scheme.acquisition_scheme import acquisition_scheme_from_bvalues
from os.path import join
import numpy as np

acquisition_path = modeling_framework.GRADIENT_TABLES_PATH
bvalues_SI = np.loadtxt(join(acquisition_path, 'bvals_hcp_wu_minn.txt')) * 1e6  # in s/m^2
gradient_directions = np.loadtxt(join(acquisition_path, 'bvecs_hcp_wu_minn.txt'))  # on unit sphere
delta = 0.0106 # in seconds
Delta = 0.0431 # in seconds
acq_scheme = acquisition_scheme_from_bvalues(
    bvalues_SI, gradient_directions, delta, Delta)

In Microstructure Imaging, microstructure models like NODDI represent the combined diffusion in the intra- and extra-axonal space as a dispersed stick and a dispersed zeppelin. NODDI further imposes two constraints on the parameters of the model:
- Certain parameters are the same between the zeppelin and the stick, such as the orientation $\mu$, parallel diffusivity $\lambda_\parallel$, and concentration $\kappa$.
- The model imposes a Tortuosity constraint, such that $\lambda_\perp=(1-vf)\lambda_\parallel$.

The microstructure toolbox allows us to straightforwardly impose such constraints on a microstructure model.  First we instantiate the required models.

In [2]:
from microstruktur.signal_models import dispersed_models
watson_stick = dispersed_models.SD1C1WatsonDispersedStick()
watson_zeppelin = dispersed_models.SD1G4WatsonDispersedZeppelin()

Just combining these two models results in a model with a lot of parameters, which will probably be very hard to optimize to a global optimum.

In [3]:
dispersed_stick_and_zeppelin = (
    modeling_framework.MultiCompartmentMicrostructureModel(
        models=[watson_stick, watson_zeppelin]))
dispersed_stick_and_zeppelin.parameter_cardinality

OrderedDict([('SD1G4WatsonDispersedZeppelin_1_mu', 2),
             ('SD1G4WatsonDispersedZeppelin_1_lambda_perp', 1),
             ('SD1C1WatsonDispersedStick_1_kappa', 1),
             ('SD1C1WatsonDispersedStick_1_lambda_par', 1),
             ('SD1G4WatsonDispersedZeppelin_1_kappa', 1),
             ('SD1C1WatsonDispersedStick_1_mu', 2),
             ('SD1G4WatsonDispersedZeppelin_1_lambda_par', 1),
             ('partial_volume_0', 1)])

We can impose the required parameter constraints by making a parameter linking list. Such a list has the general shape as follows:  
[(model1, parameter_to_be_linked, linking_function, [(model2, link_function_input_parameter)])]

For example, to make the orientation $\mu$ the same between the Stick and Zeppelin, the linking list would look like this:  
[(watson_zeppelin, 'mu',  parameter_equality, [(watson_stick, 'mu')])]

To add a tortuosity constraint, the linking list would look as follows:  
[(watson_zeppelin, 'lambda_perp', T1_tortuosity, [(None, 'partial_volume_0'), (watson_stick, 'lambda_par')])] 

Notice that T1_tortuosity takes 2 input parameters, namely the volume fraction and parallel diffusivity. We can impose a list of links/constraints by simply growing the list:

In [4]:
from microstruktur.utils.utils import (
    T1_tortuosity, parameter_equality)

parameter_links_dispersed_stick_and_tortuous_zeppelin = [
    (watson_zeppelin, 'lambda_perp', T1_tortuosity,
     [(None, 'partial_volume_0'), (watson_stick, 'lambda_par')]
    ), 
    (watson_zeppelin, 'lambda_par', parameter_equality,
     [(watson_stick, 'lambda_par')]
    ),
    (watson_zeppelin, 'mu',  parameter_equality, 
     [(watson_stick, 'mu')]
    ),
    (watson_zeppelin, 'kappa',  parameter_equality, 
     [(watson_stick, 'kappa')]
    ),
]

Now that the parameter linking is prepared, we can simply give them as input in PartialVolumeCombinedMicrostructureModel. Notice that parameter_cardinality now only shows non-linked (so the to-be-optimized) parameters. The remaining parameters are linked internally.

In [5]:
dispersed_stick_and_tortuous_zeppelin = (
    modeling_framework.MultiCompartmentMicrostructureModel(
        models=[watson_stick, watson_zeppelin],
        parameter_links=parameter_links_dispersed_stick_and_tortuous_zeppelin)
)
dispersed_stick_and_tortuous_zeppelin.parameter_cardinality

OrderedDict([('SD1C1WatsonDispersedStick_1_kappa', 1),
             ('SD1C1WatsonDispersedStick_1_lambda_par', 1),
             ('SD1C1WatsonDispersedStick_1_mu', 2),
             ('partial_volume_0', 1)])

Once the multi-compartment model is made, simulating and fitting data is exactly as in the previous tutorials.